[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/natural-voice-assistant-jupyter/blob/main/test.ipynb)

In [ ]:
%cd /content
!git clone --recursive -b dev https://github.com/camenduru/natural_voice_assistant
%cd /content/natural_voice_assistant

!apt install -qq espeak-ng portaudio19-dev aria2
!pip install -q pyaudio sounddevice wget omegaconf pytorch_lightning phonemizer munch einops_exts accelerate pyngrok
!pip install -q git+https://github.com/resemble-ai/monotonic_align

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/audo/all_test/resolve/main/ash.wav -d /content -o ash.wav

import nltk
nltk.download('punkt')

import gc, torch
from models_voice_assistant.stt_llm_tts_model import STT_LLM_TTS

def flush():
  """Flush Cuda cache to prevent side effect and slowdowns   
  """
  gc.collect()
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()

if torch.cuda.is_available():
    device = 'cuda'
    flush()
else:
    device = 'cpu'

model = STT_LLM_TTS(device=device)

In [ ]:
import nltk
nltk.download('punkt')
import gc, torch
import soundfile as sf
from models_voice_assistant.stt_llm_tts_model import STT_LLM_TTS

wav_file = '/content/ash.wav'
signal, sr = sf.read(wav_file)
signal = signal[:, 0]
chunk_size = int(8000 * 0.08)
chunks = [signal[i:i + chunk_size] for i in range(0, len(signal), chunk_size)]

In [ ]:
for chunk in chunks:
    transcription = model.call_STT(features)
    response = model.call_LLM(transcription)
    wav = model.call_TTS(response)